<a href="https://colab.research.google.com/github/tomsabag/MSc/blob/main/HebSBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODOs:
[Losses](https://sbert.net/docs/sentence_transformer/loss_overview.html)

softmax/cosinesimilarity/triplet losses/mnrl

add warmup

pooling

parameters importance test

write: abstract, intro, experiments, results, discussion


In [1]:
!pip install wandb -qU
!pip install sentence_transformers
!pip install accelerate datasets
#TODO: pip install --upgrade sentence-transformers. see if it fetches the models and not initalize new one with max pooling ...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_c

In [2]:
# Data analysis, engineering and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
import wandb
from datasets import Dataset
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr, pearsonr

# Torch
import torch
import torch.nn as nn
from torch.nn import CosineEmbeddingLoss
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.functional import cosine_similarity
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

# Sentence bert
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments, SentenceTransformerModelCardData
from sentence_transformers import models, datasets, losses, evaluation, util
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SequentialEvaluator, SimilarityFunction, TripletEvaluator

In [3]:
# Use gpu for training
def get_device() -> torch.device:
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

In [4]:
device = get_device()
device

device(type='cuda')

# Datasets

## NLI Dataset

In [5]:
# Removing unlabeled data
def clean_nli_df(df : pd.DataFrame) -> pd.DataFrame:
  df = df[df["original_label"] != '-']
  df = df.reset_index(drop=True)
  return df

In [6]:
# Load nli datasets
train_df_nli = pd.read_json("https://huggingface.co/datasets/HebArabNlpProject/HebNLI/resolve/main/HebNLI_train.jsonl", lines=True).sort_values(by="promptID")#.head(1000)
dev_df_nli = pd.read_json("https://huggingface.co/datasets/HebArabNlpProject/HebNLI/raw/main/HebNLI_val.jsonl", lines=True)
test_df_nli = pd.read_json("https://huggingface.co/datasets/HebArabNlpProject/HebNLI/raw/main/HebNLI_test.jsonl", lines=True)

In [7]:
# Create triplets data set:
# (Anchor, Positive & Negative)
def create_triplets_df(df: pd.DataFrame) -> pd.DataFrame:
  entailments = df[df.original_label == "entailment"][["translation1", "translation2"]]
  contradictions = df[df.original_label == "contradiction"][["translation1", "translation2"]]
  triplets = pd.merge(entailments, contradictions, on=["translation1"])
  triplets.rename(columns={
    'translation1': 'anchor',
    'translation2_x': 'positive',
    'translation2_y': 'negative'}, inplace=True)
  return triplets

## STS Dataset

In [8]:
def create_sts_df() -> pd.DataFrame:
  # Read and shuffle the data
  sts_df = pd.read_csv("https://drive.google.com/uc?id=1KVbqWmGV8f54P1bzltX3RAlZJSNmMI3p")
  sts_df = sts_df.sample(frac=1, random_state=42).reset_index(drop=True)

  # Split the data into test and dev sets of equal lengths
  split_index = len(sts_df) // 2
  sts_test_df = sts_df[:split_index].reset_index(drop=True)
  sts_dev_df = sts_df[split_index:].reset_index(drop=True)
  return sts_dev_df, sts_test_df

## Dataset Preprocessing

In [9]:
# Preprocess all datasets.
def data_preprocessing() -> tuple[Dataset]:
  # Nli dataset (to triplets)
  train_df_nli = pd.read_json("https://huggingface.co/datasets/HebArabNlpProject/HebNLI/resolve/main/HebNLI_train.jsonl", lines=True).sort_values(by="promptID")#.head(1000)
  # Clean
  train_df_nli = clean_nli_df(train_df_nli)

  train_df_triplets_nli = create_triplets_df(train_df_nli)
  train_df_nli, dev_df_nli = train_test_split(train_df_triplets_nli, test_size=0.1, random_state=42)

  # Sts dataset
  sts_dev_df, sts_test_df = create_sts_df()
  return Dataset.from_pandas(train_df_triplets_nli), Dataset.from_pandas(sts_dev_df), Dataset.from_pandas(dev_df_nli) ,Dataset.from_pandas(sts_test_df)

In [10]:
train_df_triplets_nli, sts_dev_df, nli_dev_df, sts_test_df = data_preprocessing()

#Model Pipeline

In [11]:
# Create one of three base models:
# Alephbert, Dictabert and Multillingual-bert
def create_model(config) -> SentenceTransformer:
  model_names = {
    "alephbert" : "onlplab/alephbert-base",
    "dictabert" : "dicta-il/dictabert",
    "mbert" : "google-bert/bert-base-multilingual-cased",
    "bert-base-nli-mean-tokens" : "bert-base-nli-mean-tokens" #TODO: large
  }
  # Create a bert model.
  # Hyperparameters, architecture and loss function are defined in a 'config' object' down below.
  bert = models.Transformer(model_names[config["bert"]]).to(device)
  pooling_model = models.Pooling(bert.get_word_embedding_dimension(),
                                 pooling_mode_mean_tokens=config.pooling_method=="mean",
                                 pooling_mode_cls_token=config.pooling_method=="cls",
                                 pooling_mode_max_tokens=config.pooling_method=="max")
  bert = SentenceTransformer(modules=[bert, pooling_model], device=device, similarity_fn_name=config["similarity_fn_name"])
  #bert = SentenceTransformer(models[config["bert"]], device=device, similarity_fn_name=config["similarity_fn_name"])
  return bert

In [12]:
# Data preprocessing, Model Architecture, Evaluation, Loss function and Training
# Are done and defined within the trainer supplier objects.
def pipeline(config): #TODO: return model ->:
  trainer_supplier = {
    "softmax" : SoftmaxTrainer,
    "cosinesimilarity" : CosineSimilarityTrainer,
    "triplet" : TripletLossTrainer,
    "mnrl" : MultipleNegativesRankingTrainer
  }
  trainer = trainer_supplier[config.trainer](config).get_trainer()
  trainer.train()
  model = trainer.model
  return model

In [13]:
#TODO: remove?
def create_loss(config: dict, model: SentenceTransformer) -> losses.MultipleNegativesRankingLoss:
  #TODO: return loss by config
  return MultipleNegativesRankingLoss(model)

In [14]:
#TODO: remove?
def create_optimizer(config: dict, model: SentenceTransformer) -> torch.optim:
  #TODO: return optimizer by config
  return Adam(model.parameters(), lr=config.learning_rate)

In [15]:
# During the training process, evaluate performence on both nli triplets, and sts dev set
def create_dev_evaluator(config, model) -> SequentialEvaluator:
  # Nli dataset. We took 10% of the training nli dataset as dev set
  # Evaluate the model during the training process on triplets of (anchor, positive, negative)
  triplets_evaluator = TripletEvaluator(
    anchors   = nli_dev_df["anchor"],
    positives = nli_dev_df["positive"],
    negatives = nli_dev_df["negative"],
    main_distance_function = config["similarity_fn_name"],
    batch_size = config["batch_size"],
    #show_progress_bar = True,
    name = "Nli")

  # Sts dataset. we took half of the sts dataset as dev set to evaluate our model during training.
  # Evaluate the model during the training process on pairs of sentences, measure similarity
  sts_evaluator = EmbeddingSimilarityEvaluator(
    sentences1 = sts_dev_df["sentence1"],
    sentences2 = sts_dev_df["sentence2"],
    scores     = sts_dev_df["score"],
    main_similarity = config["similarity_fn_name"],
    batch_size = config["batch_size"],
    #show_progress_bar = True,
    name = "Sts",
  )
  seq_evaluator = SequentialEvaluator([sts_evaluator, triplets_evaluator])
  return seq_evaluator

In [16]:
# Test the trained model on the sts dataset

def test(config, model, test_dataset: pd.DataFrame) -> None:
  # Define a test evaluator
  normalized_scores = np.array(test_dataset["score"]) / 5
  test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1 = test_dataset["sentence1"],
    sentences2 = test_dataset["sentence2"],
    scores     = normalized_scores,
    main_similarity = config.similarity_fn_name,
    batch_size=config.batch_size,
    show_progress_bar=True,
    name="Sts")
  # Test model
  print("testing now..")
  test_results = test_evaluator(model)
  print("test results:", test_results)
  wandb.log({f"test/{config['trainer']}/results": test_results})


In [17]:
# # Save the model
# def save(config, model: SentenceTransformer, run_name:str) -> None:
#   model.save_pretrained(f"models/{config.bert}/lr={config.learning_rate}/similarity_fn_name={config.similarity_fn_name}/batch_size={config.batch_size}")

In [18]:
def run_sweep(config=None):
  print("config:", config)
  # tell wandb to get started
  with wandb.init(project="NLP-final-project", config=config) as run:
    # Sweep hyperparameters
    config = wandb.config

    model = pipeline(config)

    # Testing
    test(config, model, sts_test_df)

    #save(model, run.name)

  return model

In [19]:
# Prepare nli dataset for classification based learning
# End up with pairs of sentences and a label: (Sentence1, Sentence2, Label)
# Where 'entailment', 'neutral' and 'contradiction' are mapped to 0, 1 and 2 respectively
def create_softmax_dataset(nli_dataframe):
  dataframe = clean_nli_df(nli_dataframe)
  dataframe['original_label'] = dataframe['original_label'].map({
                                                              'entailment': 0,
                                                              'neutral': 1,
                                                              'contradiction': 2
                                                              })
  dataframe = dataframe[["translation1", "translation2", "original_label"]]
  dataframe = dataframe.rename(columns={
                                    'translation1': 'sentence1',
                                    'translation2': 'sentence2',
                                    'original_label': 'label'}, inplace=False)
  return Dataset.from_pandas(dataframe)

# A 3-way softmax based model
# Use bert as a feature extractor, and a linear layer as a classifier.
# We experimented with adding 1, 2 and 3 linear layers on top as a classifier,
# But didn't get better results
class SoftmaxTrainer():
  def __init__(self, config):
    self.config = config
    self.model = create_model(config)

  def get_trainer(self):
    loss = losses.SoftmaxLoss(self.model, self.model.get_sentence_embedding_dimension(), num_labels=3) # Use cross entropy based loss
    trainer = SentenceTransformerTrainer(
        model = self.model,
        args = SentenceTransformerTrainingArguments(**(self.config["trainer_args"])),
        train_dataset = create_softmax_dataset(train_df_nli),
        eval_dataset = create_softmax_dataset(dev_df_nli),  # TODO: change dataset
        evaluator = create_dev_evaluator(self.config, self.model),
        loss = loss
    )
    return trainer




In [20]:
# Prepare nli dataset for regression based learning
# End up with pairs of sentences and a label: (Sentence1, Sentence2, Score)
# Where the similarity scores are normalized in [0, 1]
def create_cosinesimilarity_dataset(nli_dataframe):
  dataframe = clean_nli_df(nli_dataframe)
  dataframe['original_label'] = dataframe['original_label'].map({
                                                              'entailment': 1,
                                                              'neutral': 0.5,
                                                              'contradiction': 0
                                                              })
  dataframe = dataframe[["translation1", "translation2", "original_label"]]
  dataframe = dataframe.rename(columns={
                                    'translation1': 'sentence1',
                                    'translation2': 'sentence2',
                                    'original_label': 'score'}, inplace=False)
  return Dataset.from_pandas(dataframe)


# Use bert as a feature extractor, and finetune its embeddings.
# Using cosine similarity on bert's embeddings,
# Encouraging sentences which hold 'entailment' relationship closer in the latent space,
# and sentences which hold contradictive relationship further,
# Which is what the model is tested for on sts dataset
class CosineSimilarityTrainer():
  def __init__(self, config):
    self.config = config
    self.model = create_model(config)


  def get_trainer(self):
    loss = losses.CosineSimilarityLoss(self.model) # Use cosine similarity as a loss function: || score - cos(similarity(sentence1, sentence2) ||_2
    trainer = SentenceTransformerTrainer(
        model = self.model,
        args = SentenceTransformerTrainingArguments(**(self.config["trainer_args"])),
        train_dataset = create_cosinesimilarity_dataset(train_df_nli),
        eval_dataset = create_cosinesimilarity_dataset(dev_df_nli),
        evaluator = create_dev_evaluator(self.config, self.model),
        loss = loss
    )
    return trainer


In [21]:
# Prepare nli dataset for multi negative ranking loss based learning
# End up with triplets of: (Anchor, Positive, Negative)
# Where the anchor holds an 'entailment' relationship to the positive sample,
# And a 'contradiction' relationship to the negative sample
def create_contrastive_dataset(nli_dataframe):
  entailments = nli_dataframe[nli_dataframe.original_label == "entailment"][["translation1", "translation2"]]
  contradictions = nli_dataframe[nli_dataframe.original_label == "contradiction"][["translation1", "translation2"]]
  triplets = pd.merge(entailments, contradictions, on=["translation1"])
  triplets.rename(columns={
    'translation1': 'anchor',
    'translation2_x': 'positive',
    'translation2_y': 'negative'}, inplace=True)
  return Dataset.from_dict(triplets)

class TripletLossTrainer():
  def __init__(self, config):
    self.config = config
    self.model = create_model(config)


  def get_trainer(self):
    # Use triplet loss, which is a type of contrastive loss.
    # In this case, anchors are contrasted with both positive and negative samples
    # to better learn sentence representations and similarities
    loss = losses.TripletLoss(self.model)
    trainer = SentenceTransformerTrainer(
        model = self.model,
        args = SentenceTransformerTrainingArguments(**(self.config["trainer_args"])),
        train_dataset = create_contrastive_dataset(train_df_nli),
        eval_dataset = create_contrastive_dataset(dev_df_nli),
        evaluator = create_dev_evaluator(self.config, self.model),
        loss = loss
    )
    return trainer

In [22]:
# Prepare nli dataset for multi negative ranking loss based learning
# End up with triplets of: (Anchor, Positive, Negative)
# Where the anchor holds an 'entailment' relationship to the positive sample,
# And a 'contradiction' relationship to the negative sample
def create_mlnr_dataset(nli_dataframe):
  entailments = nli_dataframe[nli_dataframe.original_label == "entailment"][["translation1", "translation2"]]
  contradictions = nli_dataframe[nli_dataframe.original_label == "contradiction"][["translation1", "translation2"]]
  triplets = pd.merge(entailments, contradictions, on=["translation1"])
  triplets.rename(columns={
    'translation1': 'anchor',
    'translation2_x': 'positive',
    'translation2_y': 'negative'}, inplace=True)
  return Dataset.from_dict(triplets)

class MultipleNegativesRankingTrainer():
  def __init__(self, config):
    self.config = config
    self.model = create_model(config)


  def get_trainer(self):
    # Use multiple negative ranking loss, which is also type of contrastive loss.
    # In this case, a positive sample is contrasted with multiple negative samples,
    # And negative sample is contrasted with multiple positive samples to better learn
    # sentence representations and similarities
    loss = losses.MultipleNegativesRankingLoss(self.model)
    trainer = SentenceTransformerTrainer(
        model = self.model,
        args = SentenceTransformerTrainingArguments(**(self.config["trainer_args"])),
        train_dataset = create_mlnr_dataset(train_df_nli),
        eval_dataset = create_mlnr_dataset(dev_df_nli),
        evaluator = create_dev_evaluator(self.config, self.model),
        loss = loss
    )
    return trainer


In [ ]:
'''config = {
    "method": "bayes", #TODO: bayesian
    "metric": {"goal": "maximize", "name": "Dev/Sts Spearman Cosine"},
    "parameters": {
        "trainer": {"values": ["softmax", "cosinesimilarity", "triplet", "mnrl"]},
        "similarity_fn_name": {"values": ["cosine", "dot", "euclidean", "manhattan"]},
        "epochs":{"value": 1},
        "batch_size": {"value": 16},
        "bert": {"values": ["mbert", "dictabert","dictabert"]},
        "trainer_args": {
            "parameters" : {
              "output_dir" : {"value": "models"},
              "learning_rate" : {"min": 1e-7, "max": 1e-4},
              "num_train_epochs": {"value": 1},
              "per_device_train_batch_size": {"values": [8, 16]},
              "per_device_eval_batch_size": {"value" : 16},
              "warmup_ratio": {"value": 0.1},
              "eval_strategy":{"value":"steps"},
              "eval_steps": {"value":500},
              "save_strategy": {"value":"steps"},
              "save_steps": {"value":500},
              "save_total_limit":{"value":1},
              "logging_steps":{"value":100}
          }
        }
    }
}

# Use weights and biases to monitor training process,
# And finetune the network experimenting with different hyperparameters
sweep_id = wandb.sweep(sweep=config, project="NLP-final-project")
#sweep_id = "qai1hafw"
wandb.agent(sweep_id, function=lambda: run_sweep(config), count=10)'''

'config = {\n    "method": "bayes", #TODO: bayesian\n    "metric": {"goal": "maximize", "name": "Dev/Sts Spearman Cosine"},\n    "parameters": {\n        "trainer": {"values": ["softmax", "cosinesimilarity", "triplet", "mnrl"]},\n        "similarity_fn_name": {"values": ["cosine", "dot", "euclidean", "manhattan"]},\n        "epochs":{"value": 1},\n        "batch_size": {"value": 16},\n        "bert": {"values": ["mbert", "dictabert","dictabert"]},\n        "trainer_args": {\n            "parameters" : {\n              "output_dir" : {"value": "models"},\n              "learning_rate" : {"min": 1e-7, "max": 1e-4},\n              "num_train_epochs": {"value": 1},\n              "per_device_train_batch_size": {"values": [8, 16]},\n              "per_device_eval_batch_size": {"value" : 16},\n              "warmup_ratio": {"value": 0.1},\n              "eval_strategy":{"value":"steps"},\n              "eval_steps": {"value":500},\n              "save_strategy": {"value":"steps"},\n         

# Training and evaluation of best model architecture for 3 epochs

**Model**: Dictabert

**Loss function**: MNRL

**Epochs**: 3

**Learning rates**: grid search $\in [2*10^{-6}, 7*10^{-6}, 2*10^{-5}]$


In [27]:
config = {
    "method": "grid",
    "metric": {"goal": "maximize", "name": "eval/Sts_spearman_cosine"},
    "parameters": {
        "trainer": {"values": ["mnrl"]},
        "similarity_fn_name": {"values": ["cosine"]},
        "epochs":{"value": 3},
        "batch_size": {"value": 16},
        "pooling_method": {"value": "mean"},
        "bert": {"values": ["dictabert"]},
        "trainer_args": {
            "parameters" : {
              "output_dir" : {"value": "models"},
              "learning_rate" : {"values" : [2e-6, 7e-6, 2e-5]},
              "num_train_epochs": {"value": 3},
              "per_device_train_batch_size": {"value": 16},
              "per_device_eval_batch_size": {"value" : 16},
              "warmup_ratio": {"value": 0.1},
              "eval_strategy":{"value":"steps"},
              "eval_steps": {"value":500},
              "save_strategy": {"value":"steps"},
              "save_steps": {"value":5000},
              "save_total_limit":{"value":1},
              "logging_steps":{"value":500}
          }
        }
    }
}

# Use weights and biases to monitor training process,
# And finetune the network experimenting with different hyperparameters
sweep_id = wandb.sweep(sweep=config, project="NLP-final-project")
#sweep_id = "qai1hafw"
wandb.agent(sweep_id, function=lambda: run_sweep(config), count=1)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: vfmm84g6
Sweep URL: https://wandb.ai/HebSBert/NLP-final-project/sweeps/vfmm84g6


...

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

test results: {'Sts_pearson_cosine': 0.79561475926572, 'Sts_spearman_cosine': 0.790537848779515, 'Sts_pearson_manhattan': 0.7895269298550275, 'Sts_spearman_manhattan': 0.7820546040795404, 'Sts_pearson_euclidean': 0.7909038254191036, 'Sts_spearman_euclidean': 0.7827202382669715, 'Sts_pearson_dot': 0.7365371164293609, 'Sts_spearman_dot': 0.724092815039966, 'Sts_pearson_max': 0.79561475926572, 'Sts_spearman_max': 0.790537848779515}


eval/Nli_cosine_accuracy,▁▃▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████████████
eval/Nli_dot_accuracy,█▆▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/Nli_euclidean_accuracy,▁▃▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████████████
eval/Nli_manhattan_accuracy,▁▃▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████████████
eval/Nli_max_accuracy,▁▃▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████████████
eval/Sts_pearson_cosine,▁▄▆▇▇▇█████▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇
eval/Sts_pearson_dot,▁▃▅▆▇▇▇▇▇▇▇███████████████████████████
eval/Sts_pearson_euclidean,▁▄▆▇▇▇██████▇▇█▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇██▇██
eval/Sts_pearson_manhattan,▁▄▆▇▇▇██████▇▇█▇▇▇▇▇▇▇▇▇▇█▇▇██▇▇██████
eval/Sts_pearson_max,▁▄▆▇▇▇█████▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇
eval/Sts_spearman_cosine,▁▃▅▇▇▇██████▇▇███▇█▇█▇█▇▇█████████████


wandb: Ctrl + C detected. Stopping sweep.


# Training and evaluation of model architecture

**Models**: grid search over Dictabert, Alephbert, Mbert

**Loss functions**: MNRL

**Epochs**: 1

**Learning rate**:$ [2*10^{-5}]$


In [23]:
config = {
    "method": "grid",
    "metric": {"goal": "maximize", "name": "eval/Sts_spearman_cosine"},
    "parameters": {
        "trainer": {"value":"mnrl"},
        "similarity_fn_name": {"value": "cosine"},
        "epochs":{"value": 1},
        "batch_size": {"value": 16},
        "pooling_method": {"value": "mean"},
        "bert": {"values": ["dictabert", "mbert", "alephbert"]},
        "trainer_args": {
            "parameters" : {
              "output_dir" : {"value": "models"},
              "learning_rate" : {"value" : 2e-5},
              "num_train_epochs": {"value": 1},
              "per_device_train_batch_size": {"value": 16},
              "per_device_eval_batch_size": {"value" : 16},
              "warmup_ratio": {"value": 0.1},
              "eval_strategy":{"value":"steps"},
              "eval_steps": {"value":500},
              "save_strategy": {"value":"steps"},
              "save_steps": {"value":5000},
              "save_total_limit":{"value":1},
              "logging_steps":{"value":500}
          }
        }
    }
}

# Use weights and biases to monitor training process,
# And finetune the network experimenting with different hyperparameters
sweep_id = wandb.sweep(sweep=config, project="NLP-final-project")
#sweep_id = "qai1hafw"
wandb.agent(sweep_id, function=lambda: run_sweep(config), count=3)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: aboshe7x
Sweep URL: https://wandb.ai/HebSBert/NLP-final-project/sweeps/aboshe7x


...

testing now..


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

test results: {'Sts_pearson_cosine': 0.753397687885077, 'Sts_spearman_cosine': 0.7434463045634541, 'Sts_pearson_manhattan': 0.7489888308648115, 'Sts_spearman_manhattan': 0.7385632813704458, 'Sts_pearson_euclidean': 0.7495414111212155, 'Sts_spearman_euclidean': 0.7385803432679975, 'Sts_pearson_dot': 0.6495364140091465, 'Sts_spearman_dot': 0.6405153662869684, 'Sts_pearson_max': 0.753397687885077, 'Sts_spearman_max': 0.7434463045634541}


eval/Nli_cosine_accuracy,▁▃▄▅▆▆▆▇▇███
eval/Nli_dot_accuracy,█▆▅▄▃▃▂▂▂▁▁▁
eval/Nli_euclidean_accuracy,▁▃▃▅▆▆▇▇▇███
eval/Nli_manhattan_accuracy,▁▃▃▅▆▆▇▇▇███
eval/Nli_max_accuracy,▁▃▄▅▆▆▆▇▇███
eval/Sts_pearson_cosine,▇▅▅▁▅▃▄█▆▇▇▆
eval/Sts_pearson_dot,█▂▆▁▆▄▄▇▇▇▆▅
eval/Sts_pearson_euclidean,▁▃▁▁▃▃▅█▇▆▇▇
eval/Sts_pearson_manhattan,▁▂▁▁▃▃▅█▇▆▇▇
eval/Sts_pearson_max,▄▃▁▁▃▃▅█▇▆▇▇
eval/Sts_spearman_cosine,▄▅▄▁▅▃▅█▇▇▇▇


# Training and evaluation of loss function

**Model**: Dictabert

**Loss functions**: Grid search over Softmax, CosineEmbedding, Triplet & MNRL

**Epochs**: 3

**Learning rate**:$[2*10^{-5}]$


In [28]:
config = {
    "method": "grid",
    "metric": {"goal": "maximize", "name": "eval/Sts_spearman_cosine"},
    "parameters": {
        "trainer": {"values": ["softmax", "cosinesimilarity", "triplet", "mnrl"]},
        "similarity_fn_name": {"value": "cosine"},
        "epochs":{"value": 1},
        "batch_size": {"value": 16},
        "pooling_method": {"value": "mean"},
        "bert": {"value": "dictabert"},
        "trainer_args": {
            "parameters" : {
              "output_dir" : {"value": "models"},
              "learning_rate" : {"value" : 2e-5},
              "num_train_epochs": {"value": 1},
              "per_device_train_batch_size": {"value": 16},
              "per_device_eval_batch_size": {"value" : 16},
              "warmup_ratio": {"value": 0.1},
              "eval_strategy":{"value":"steps"},
              "eval_steps": {"value":500},
              "save_strategy": {"value":"steps"},
              "save_steps": {"value":5000},
              "save_total_limit":{"value":1},
              "logging_steps":{"value":500}
          }
        }
    }
}

# Use weights and biases to monitor training process,
# And finetune the network experimenting with different hyperparameters
sweep_id = wandb.sweep(sweep=config, project="NLP-final-project")
#sweep_id = "qai1hafw"
wandb.agent(sweep_id, function=lambda: run_sweep(config), count=4)

Create sweep with ID: lcnrzcaq
Sweep URL: https://wandb.ai/HebSBert/NLP-final-project/sweeps/lcnrzcaq


wandb: Agent Starting Run: ycq3udhf with config:
wandb: 	batch_size: 16
wandb: 	bert: dictabert
wandb: 	epochs: 1
wandb: 	pooling_method: mean
wandb: 	similarity_fn_name: cosine
wandb: 	trainer: softmax
wandb: 	trainer_args: {'eval_steps': 500, 'eval_strategy': 'steps', 'learning_rate': 2e-05, 'logging_steps': 500, 'num_train_epochs': 1, 'output_dir': 'models', 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'save_steps': 5000, 'save_strategy': 'steps', 'save_total_limit': 1, 'warmup_ratio': 0.1}


config: {'method': 'grid', 'metric': {'goal': 'maximize', 'name': 'eval/Sts_spearman_cosine'}, 'parameters': {'trainer': {'values': ['softmax', 'cosinesimilarity', 'triplet', 'mnrl']}, 'similarity_fn_name': {'value': 'cosine'}, 'epochs': {'value': 1}, 'batch_size': {'value': 16}, 'pooling_method': {'value': 'mean'}, 'bert': {'value': 'dictabert'}, 'trainer_args': {'parameters': {'output_dir': {'value': 'models'}, 'learning_rate': {'value': 2e-05}, 'num_train_epochs': {'value': 1}, 'per_device_train_batch_size': {'value': 16}, 'per_device_eval_batch_size': {'value': 16}, 'warmup_ratio': {'value': 0.1}, 'eval_strategy': {'value': 'steps'}, 'eval_steps': {'value': 500}, 'save_strategy': {'value': 'steps'}, 'save_steps': {'value': 5000}, 'save_total_limit': {'value': 1}, 'logging_steps': {'value': 500}}}}}


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

testing now..


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

test results: {'Sts_pearson_cosine': 0.7932420023036347, 'Sts_spearman_cosine': 0.7910961531105941, 'Sts_pearson_manhattan': 0.7871293529984481, 'Sts_spearman_manhattan': 0.7828040049951362, 'Sts_pearson_euclidean': 0.7884106120206235, 'Sts_spearman_euclidean': 0.7839322019573002, 'Sts_pearson_dot': 0.735430211604571, 'Sts_spearman_dot': 0.7244337406616848, 'Sts_pearson_max': 0.7932420023036347, 'Sts_spearman_max': 0.7910961531105941}


eval/Nli_cosine_accuracy,▁▃▄▅▆▆▆▇▇███
eval/Nli_dot_accuracy,█▆▅▄▃▃▃▂▂▁▁▁
eval/Nli_euclidean_accuracy,▁▃▄▅▅▆▆▇▇███
eval/Nli_manhattan_accuracy,▁▃▃▅▆▆▆▇▇███
eval/Nli_max_accuracy,▁▃▄▅▆▆▆▇▇███
eval/Sts_pearson_cosine,▇▁█▁▁▃▄█▂▅▄▄
eval/Sts_pearson_dot,▄▃▆▁▆▆▅█▅▆▆▆
eval/Sts_pearson_euclidean,▇▁█▅▃▄▆█▅▆▅▅
eval/Sts_pearson_manhattan,▇▁█▅▃▄▆▇▄▆▅▅
eval/Sts_pearson_max,▇▁█▅▃▄▆█▄▆▅▅
eval/Sts_spearman_cosine,▅▁█▃▃▄▅█▃▆▅▅


# Evaluation of the importance of pooling method

In [24]:
config = {
    "method": "grid",
    "metric": {"goal": "maximize", "name": "eval/Sts_spearman_cosine"},
    "parameters": {
        "trainer": {"value": "mnrl"},
        "similarity_fn_name": {"value": "cosine"},
        "epochs":{"value": 1},
        "batch_size": {"value": 16},
        "pooling_method": {"values": ["mean", "cls", "max"]},
        "bert": {"value": "dictabert"},
        "trainer_args": {
            "parameters" : {
              "output_dir" : {"value": "models"},
              "learning_rate" : {"value" : 2e-5},
              "num_train_epochs": {"value": 1},
              "per_device_train_batch_size": {"value": 16},
              "per_device_eval_batch_size": {"value" : 16},
              "warmup_ratio": {"value": 0.1},
              "eval_strategy":{"value":"steps"},
              "eval_steps": {"value":500},
              "save_strategy": {"value":"steps"},
              "save_steps": {"value":5000},
              "save_total_limit":{"value":1},
              "logging_steps":{"value":500}
          }
        }
    }
}

# Use weights and biases to monitor training process,
# And finetune the network experimenting with different hyperparameters
sweep_id = wandb.sweep(sweep=config, project="NLP-final-project")
#sweep_id = "qai1hafw"
wandb.agent(sweep_id, function=lambda: run_sweep(config), count=3)

Create sweep with ID: 9dj5p9vl
Sweep URL: https://wandb.ai/HebSBert/NLP-final-project/sweeps/9dj5p9vl


wandb: Agent Starting Run: xof3t3lt with config:
wandb: 	batch_size: 16
wandb: 	bert: dictabert
wandb: 	epochs: 1
wandb: 	pooling_method: mean
wandb: 	similarity_fn_name: cosine
wandb: 	trainer: mnrl
wandb: 	trainer_args: {'eval_steps': 500, 'eval_strategy': 'steps', 'learning_rate': 2e-05, 'logging_steps': 500, 'num_train_epochs': 1, 'output_dir': 'models', 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'save_steps': 5000, 'save_strategy': 'steps', 'save_total_limit': 1, 'warmup_ratio': 0.1}


config: {'method': 'grid', 'metric': {'goal': 'maximize', 'name': 'eval/Sts_spearman_cosine'}, 'parameters': {'trainer': {'value': 'mnrl'}, 'similarity_fn_name': {'value': 'cosine'}, 'epochs': {'value': 1}, 'batch_size': {'value': 16}, 'pooling_method': {'values': ['mean', 'cls', 'max']}, 'bert': {'value': 'dictabert'}, 'trainer_args': {'parameters': {'output_dir': {'value': 'models'}, 'learning_rate': {'value': 2e-05}, 'num_train_epochs': {'value': 1}, 'per_device_train_batch_size': {'value': 16}, 'per_device_eval_batch_size': {'value': 16}, 'warmup_ratio': {'value': 0.1}, 'eval_strategy': {'value': 'steps'}, 'eval_steps': {'value': 500}, 'save_strategy': {'value': 'steps'}, 'save_steps': {'value': 5000}, 'save_total_limit': {'value': 1}, 'logging_steps': {'value': 500}}}}}


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

testing now..


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

test results: {'Sts_pearson_cosine': 0.7844997571545432, 'Sts_spearman_cosine': 0.7864185843262981, 'Sts_pearson_manhattan': 0.7881926441665078, 'Sts_spearman_manhattan': 0.7833781587894545, 'Sts_pearson_euclidean': 0.7875443679789712, 'Sts_spearman_euclidean': 0.7826657834656103, 'Sts_pearson_dot': 0.783667409011653, 'Sts_spearman_dot': 0.7842379130183798, 'Sts_pearson_max': 0.7881926441665078, 'Sts_spearman_max': 0.7864185843262981}


eval/Nli_cosine_accuracy,▁▃▄▅▆▆▇▇▇███
eval/Nli_dot_accuracy,█▆▅▄▃▂▂▂▁▁▁▁
eval/Nli_euclidean_accuracy,▁▃▄▅▆▆▇▇▇███
eval/Nli_manhattan_accuracy,▁▃▄▅▆▆▇▇▇███
eval/Nli_max_accuracy,▁▃▄▅▆▆▇▇▇███
eval/Sts_pearson_cosine,▂▁▇▅▄▄▆█▅▆▅▆
eval/Sts_pearson_dot,▂▁▇▄▃▅▆█▅▆▅▆
eval/Sts_pearson_euclidean,▁▁█▆▄▅▆█▄▇▄▆
eval/Sts_pearson_manhattan,▂▁█▆▄▅▆█▅▇▅▆
eval/Sts_pearson_max,▂▁▇▅▄▄▆█▅▆▅▆
eval/Sts_spearman_cosine,▁▂▇▆▅▅▇█▆▇▆▇


# Evaluation of the importance of learning rate

In [24]:
config = {
    "method": "grid",
    "metric": {"goal": "maximize", "name": "eval/Sts_spearman_cosine"},
    "parameters": {
        "trainer": {"value": "mnrl"},
        "similarity_fn_name": {"value": "cosine"},
        "epochs":{"value": 1},
        "batch_size": {"value": 16},
        "pooling_method": {"value": "mean"},
        "bert": {"value": "dictabert"},
        "trainer_args": {
            "parameters" : {
              "output_dir" : {"value": "models"},
              "learning_rate" : {"values" : [2e-4, 2e-5, 2e-8]},
              "num_train_epochs": {"value": 1},
              "per_device_train_batch_size": {"value": 16},
              "per_device_eval_batch_size": {"value" : 16},
              "warmup_ratio": {"value": 0.1},
              "eval_strategy":{"value":"steps"},
              "eval_steps": {"value":500},
              "save_strategy": {"value":"steps"},
              "save_steps": {"value":5000},
              "save_total_limit":{"value":1},
              "logging_steps":{"value":500}
          }
        }
    }
}

# Use weights and biases to monitor training process,
# And finetune the network experimenting with different hyperparameters
sweep_id = wandb.sweep(sweep=config, project="NLP-final-project")
#sweep_id = "qai1hafw"
wandb.agent(sweep_id, function=lambda: run_sweep(config), count=3)

Create sweep with ID: umu2xrki
Sweep URL: https://wandb.ai/HebSBert/NLP-final-project/sweeps/umu2xrki


wandb: Agent Starting Run: 3shq8cgo with config:
wandb: 	batch_size: 16
wandb: 	bert: dictabert
wandb: 	epochs: 1
wandb: 	pooling_method: mean
wandb: 	similarity_fn_name: cosine
wandb: 	trainer: mnrl
wandb: 	trainer_args: {'eval_steps': 500, 'eval_strategy': 'steps', 'learning_rate': 0.0002, 'logging_steps': 500, 'num_train_epochs': 1, 'output_dir': 'models', 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'save_steps': 5000, 'save_strategy': 'steps', 'save_total_limit': 1, 'warmup_ratio': 0.1}


config: {'method': 'grid', 'metric': {'goal': 'maximize', 'name': 'eval/Sts_spearman_cosine'}, 'parameters': {'trainer': {'value': 'mnrl'}, 'similarity_fn_name': {'value': 'cosine'}, 'epochs': {'value': 1}, 'batch_size': {'value': 16}, 'pooling_method': {'value': 'mean'}, 'bert': {'value': 'dictabert'}, 'trainer_args': {'parameters': {'output_dir': {'value': 'models'}, 'learning_rate': {'values': [0.0002, 2e-05, 2e-08]}, 'num_train_epochs': {'value': 1}, 'per_device_train_batch_size': {'value': 16}, 'per_device_eval_batch_size': {'value': 16}, 'warmup_ratio': {'value': 0.1}, 'eval_strategy': {'value': 'steps'}, 'eval_steps': {'value': 500}, 'save_strategy': {'value': 'steps'}, 'save_steps': {'value': 5000}, 'save_total_limit': {'value': 1}, 'logging_steps': {'value': 500}}}}}


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

testing now..


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

test results: {'Sts_pearson_cosine': 0.7052239050394219, 'Sts_spearman_cosine': 0.7066636563362375, 'Sts_pearson_manhattan': 0.7001065797818615, 'Sts_spearman_manhattan': 0.6930022339359652, 'Sts_pearson_euclidean': 0.7001058661834826, 'Sts_spearman_euclidean': 0.6934368961330774, 'Sts_pearson_dot': 0.44867442507757327, 'Sts_spearman_dot': 0.4452585958751602, 'Sts_pearson_max': 0.7052239050394219, 'Sts_spearman_max': 0.7066636563362375}


eval/Nli_cosine_accuracy,▁▂▃▅▅▆▇▇████
eval/Nli_dot_accuracy,█▆▅▄▃▂▂▂▁▁▁▁
eval/Nli_euclidean_accuracy,▁▂▂▃▃▄▅▆▆▇▇█
eval/Nli_manhattan_accuracy,▁▃▄▆▆▆▆▆▇███
eval/Nli_max_accuracy,▁▂▃▅▅▆▇▇████
eval/Sts_pearson_cosine,▁▂▃▄▅▆▇▇▇███
eval/Sts_pearson_dot,▁▂▃▄▅▆▇▇▇███
eval/Sts_pearson_euclidean,▁▂▃▄▅▆▇▇▇███
eval/Sts_pearson_manhattan,▁▂▃▄▅▆▇▇▇███
eval/Sts_pearson_max,▁▂▃▄▅▆▇▇▇███
eval/Sts_spearman_cosine,▁▂▃▄▅▆▇▇▇███
